In [ ]:
import gurobipy as grb
from gurobipy import GRB
import networkx as nx

In [ ]:
import json

space_age = json.load(open("data/space-age.json"))



In [ ]:
G = nx.DiGraph()
from collections import defaultdict


machines = defaultdict(set)
for n, f in space_age["furnace"].items():
    for c in f["crafting_categories"]:
        machines[c].add(("furnace", n))

for n, f in space_age["assembling-machine"].items():
    for c in f["crafting_categories"]:
        machines[c].add(("assembling-machine", n))


modules = defaultdict(dict)
for n, m in space_age["module"].items():
    modules[m["category"]][m["name"]] = m["effect"]

print(modules)

max_quality_level = 0
for n, q in space_age["quality"].items():
    max_quality_level = max(max_quality_level, q["level"])

items = set()

for n, r in space_age["recipe"].items():
    if "ingredients" in r:
        items.update((i["type"], i["name"]) for i in r["ingredients"])
    if "results" in r:
        items.update((i["type"], i["name"]) for i in r["results"])


In [ ]:

from itertools import chain, combinations_with_replacement

all_crafting_variables = set()
all_item_variables = set()

ingredients_to_recipes = defaultdict(set)
products_to_recipes = defaultdict(set)


def get_effect(chosen_modules):
    effects = {
        "consumption": 1.0,
        "speed": 1.0,
        "productivity": 0.0,
        "pollution": 1.0,
        "quality": 0.0,
    }

    for m in chosen_modules:
        module = space_age["module"][m]
        for k in effects:
            effects[k] += module["effect"].get(k, 0)
    
    effects["speed"] = max(effects["speed"], 0.2)
    effects["consumption"] = max(effects["consumption"], 0.2)
    effects["productivity"] = max(effects["productivity"], 0.0)
    effects["pollution"] = max(effects["pollution"], 0.2)
    effects["quality"] = max(effects["quality"], 0.)

    return effects

for _, r in space_age["recipe"].items():

    allow_quality = int(r.get("allow_quality", True))
    for q in range(allow_quality * (max_quality_level+1)):

        for i in r.get("ingredients", []):
            all_item_variables.add((i["type"], i["name"], 0 if i["type"] == "fluid" else q))
        for i in r.get("results", []):
            all_item_variables.add((i["type"], i["name"], 0 if i["type"] == "fluid" else q))


        for m in machines[r.get("category", None)]:
            machine = space_age[m[0]][m[1]]

            slots = machine.get("module_slots", 0)
            possible_modules_categories = machine.get("allowed_module_categories", modules.keys())
            possible_modules = set().union(*(set(modules[c].keys()) for c in possible_modules_categories))

            possible_modules = {m for m in possible_modules if space_age["module"][m].get("tier", 1) == 2}

        
            for chosen_modules in chain.from_iterable(combinations_with_replacement(possible_modules, i) for i in range(slots)):

                key = (r["name"], q, m, chosen_modules)
                all_crafting_variables.add(key)

                effects = get_effect(chosen_modules)

                for k, i in enumerate(r.get("ingredients", [])):
                    ingredients_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else q)].add((key, k))
                for k, i in enumerate(r.get("results", [])):
                    if effects["quality"] > 0:
                        for qual in range(q, max_quality_level+1):
                            products_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else qual)].add((key, k))
                    products_to_recipes[(i["type"], i["name"], 0 if i["type"] == "fluid" else q)].add((key, k))


In [ ]:
model = grb.Model()

total_machines = model.addVars(all_crafting_variables, obj=10, vtype=GRB.INTEGER)
machine_usage = model.addVars(all_crafting_variables)
used = model.addVars(all_item_variables, name="used")
produced = model.addVars(all_item_variables, obj=1, name="produced")

produced["item", "scrap", 0].setAttr(GRB.Attr.LB, 1)

# flow feasibility
model.addConstrs(used[i] <= produced[i] for i in all_item_variables)

# capacity constraints: machine_usage = crafts per second
for (rname, q, m, mods) in all_crafting_variables:
    machine = space_age[m[0]][m[1]]
    recipe = space_age["recipe"][rname]

    base_speed = machine.get("crafting_speed", 1.0)
    craft_time = recipe.get("energy_required", 0.5)  # seconds at speed 1

    effects = get_effect(mods)
    speed_mult = effects.get("speed", 1.0)

    eff_speed = base_speed * speed_mult
    crafts_per_second = eff_speed / craft_time

    model.addConstr(
        machine_usage[(rname, q, m, mods)]
        <= total_machines[(rname, q, m, mods)] * crafts_per_second
    )

alpha = 0.1  # 10% chance to upgrade again

def quality_prob(base_q, target_q, Q):
    # No quality modules or targeting below base quality
    if Q <= 0.0 or target_q < base_q:
        return 1.0 if target_q == base_q else 0.0

    if Q > 1.0:
        Q = 1.0

    # Already at max quality: can't go higher
    if base_q >= max_quality_level:
        return 1.0 if target_q == base_q else 0.0

    # Stay at base quality: no upgrade at all
    if target_q == base_q:
        return 1.0 - Q

    # Number of upgrade steps
    steps = target_q - base_q

    # Intermediate tiers: base_q < target_q < max_quality_level
    if target_q < max_quality_level:
        # P = Q * α^(steps-1) * (1-α)
        return Q * (alpha ** (steps - 1)) * (1.0 - alpha)

    # Top tier: target_q == max_quality_level
    # P = Q * α^(steps-1)
    if target_q == max_quality_level:
        return Q * (alpha ** (steps - 1))

    return 0.0

for item_key in all_item_variables:
    # ----- used -----
    expr_used = grb.LinExpr()
    for (rname, q, m, mods), k in ingredients_to_recipes.get(item_key, []):
        r = space_age["recipe"][rname]
        ing = r.get("ingredients", [])[k]
        amt = ing.get("amount", 0)
        if amt:
            expr_used.addTerms(amt, machine_usage[(rname, q, m, mods)])

    model.addConstr(used[item_key] == expr_used)

    # ----- produced -----
    expr_prod = grb.LinExpr()
    itype, iname, item_quality = item_key

    for (rname, q, m, mods), k in products_to_recipes.get(item_key, []):
        r = space_age["recipe"][rname]
        res = r.get("results", [])[k]

        # base amount from amount / amount_min / amount_max
        if "amount" in res:
            min_amt = res["amount"]
            max_amt = res["amount"]
        else:
            min_amt = res.get("amount_min", 0)
            max_amt = res.get("amount_max", min_amt)
            if max_amt < min_amt:
                max_amt = min_amt
        base = 0.5 * (min_amt + max_amt)

        prob = res.get("probability", 1.0)
        extra = res.get("extra_count_fraction", 0.0)
        spoiled = res.get("percent_spoiled", 0.0)

        effects = get_effect(mods)
        Q = effects.get("quality", 0.0)
        if Q < 0.0:
            Q = 0.0
        elif Q > 1.0:
            Q = 1.0

        prod_bonus = effects.get("productivity", 0.0)
        # respect recipe allow_productivity and maximum_productivity
        if not r.get("allow_productivity", False):
            prod_bonus = 0.0
        else:
            max_prod = r.get("maximum_productivity", 3.0)
            if prod_bonus < 0.0:
                prod_bonus = 0.0
            if prod_bonus > max_prod:
                prod_bonus = max_prod

        # base expected amount per craft, with productivity and spoilage
        expected_base = (prob * base + extra) * (1.0 + prod_bonus) * (1.0 - spoiled)
        if expected_base == 0.0:
            continue

        if res["type"] == "fluid":
            # fluids ignore quality: all fluid is quality 0
            if item_quality != 0:
                continue
            expected = expected_base
        else:
            # items: distribute over qualities
            expected = expected_base * quality_prob(q, item_quality, Q)

        if expected:
            expr_prod.addTerms(expected, machine_usage[(rname, q, m, mods)])

    # external sources/sinks
    if item_key in [("item", "scrap", 0), ("fluid", "heavy-oil", 0)]:
        expr_prod.addConstant(1000)

    if item_key in [("item", "quality-module", 5)]:
        expr_prod.addConstant(-1)

    model.addConstr(produced[item_key] == expr_prod)

model.setParam(GRB.Param.MIPGap, 0.01)
model.optimize()

for machine_key in all_crafting_variables:
    if total_machines[machine_key].X > 0:
        print(machine_key, total_machines[machine_key].X)


In [ ]:
c = defaultdict(float)
for machine_key in all_crafting_variables:
    if total_machines[machine_key].X > 0:
        c[machine_key[2]] += total_machines[machine_key].X
        for m in machine_key[3]:
            c[m] += total_machines[machine_key].X


display(c)
